# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Crohn's Disease"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Lower_Grade_Glioma_(LGG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_LGG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'additional_surgery_metastatic_procedure',
 'age_at_initial_pathologic_diagnosis']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', '

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0],
 'days_to_birth': [-11509.0, -24578.0, -16297.0, -13565.0, -18494.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, 234.0, 1335.0, 1106.0, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [3574.0, nan, nan, 481.0, 1828.0],
 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_of_animal_insect_allergy': ['12 - 20 Years',
  nan,
  nan,
  nan,
  nan],
 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2005.0,
  2005.0,
  2006.0,
  2009.0,
  2010.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lower Grade Glioma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-CS-4938-01,1,31.0,0.0,1.675408,2.256574,-0.025535,-0.129672,0.512522,1.00569,0.980606,...,2.145814,-0.086682,-0.748878,-0.411617,0.508467,-1.28139,0.587923,-1.931327,-1.200995,-5.525533
TCGA-CS-4941-01,1,67.0,1.0,1.677108,1.531874,-0.103335,-0.044972,1.044022,0.68629,0.715906,...,3.563814,-0.086682,2.294922,2.043683,0.703167,-1.28139,-0.211777,-0.691827,-0.067395,-4.274433
TCGA-CS-4942-01,1,44.0,0.0,1.451408,0.339574,0.227665,-0.359972,0.479422,0.83969,0.928806,...,2.061014,-0.086682,-0.748878,2.006583,1.287867,-1.28139,-0.521377,-1.996927,-1.020895,-5.096833
TCGA-CS-4943-01,1,37.0,1.0,1.159508,1.747574,0.193265,0.049128,0.345022,0.20049,0.736706,...,-0.864186,-0.086682,-0.748878,-0.862817,0.231767,-1.28139,0.760023,-1.260727,-2.100995,-4.541333
TCGA-CS-4944-01,1,50.0,1.0,0.847608,3.430774,-0.531035,-0.503272,0.820222,1.21089,0.934706,...,3.457214,-0.086682,-0.159678,1.123883,0.491667,-1.28139,-0.713077,-1.252627,-1.109495,-3.516133


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 529 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lower Grade Glioma', the least common label is '1' with 529 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lower Grade Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 32.0
  50% (Median): 41.0
  75%: 53.0
Min: 14.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 238 occurrences. This represents 44.99% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Crohn's-Disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE123088',
 'GSE66407',
 'GSE169568',
 'GSE207022',
 'GSE179285',
 'GSE186582',
 'GSE123086']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE123086"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

("/content/drive/MyDrive/AI4Science_Public/data/GEO/Crohn's-Disease/GSE123086/GSE123086_family.soft.gz",
 "/content/drive/MyDrive/AI4Science_Public/data/GEO/Crohn's-Disease/GSE123086/GSE123086_series_matrix.txt.gz")

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 127 and 39, respectively)

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,...,GSM3495040,GSM3495041,GSM3495042,GSM3495043,GSM3495044,GSM3495045,GSM3495046,GSM3495047,GSM3495048,GSM3495049
0,!Sample_characteristics_ch1,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,...,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells
1,!Sample_characteristics_ch1,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,...,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS,primary diagnosis: ULCERATIVE_COLITIS
2,!Sample_characteristics_ch1,Sex: Male,diagnosis2: ATOPIC_ECZEMA,Sex: Female,Sex: Female,Sex: Male,Sex: Male,diagnosis2: ATHEROSCLEROSIS,Sex: Male,Sex: Female,...,Sex: Male,Sex: Male,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Male
3,!Sample_characteristics_ch1,age: 56,Sex: Male,age: 20,age: 51,age: 37,age: 61,Sex: Male,age: 31,age: 56,...,age: 20,age: 69,age: 69,age: 47,age: 47,age: 56,age: 54,age: 53,age: 50,age: 22
4,!Sample_characteristics_ch1,NaN,age: 63,NaN,NaN,NaN,NaN,age: 74,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell type: CD4+ T cells'],
 1: ['primary diagnosis: ASTHMA',
  'primary diagnosis: ATHEROSCLEROSIS',
  'primary diagnosis: BREAST_CANCER',
  'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA',
  'primary diagnosis: CROHN_DISEASE',
  'primary diagnosis: ATOPIC_ECZEMA',
  'primary diagnosis: HEALTHY_CONTROL',
  'primary diagnosis: INFLUENZA',
  'primary diagnosis: OBESITY',
  'primary diagnosis: PSORIASIS',
  'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS',
  'primary diagnosis: TYPE_1_DIABETES',
  'primary diagnosis: ACUTE_TONSILLITIS',
  'primary diagnosis: ULCERATIVE_COLITIS'],
 2: ['Sex: Male',
  'diagnosis2: ATOPIC_ECZEMA',
  'Sex: Female',
  'diagnosis2: ATHEROSCLEROSIS',
  'diagnosis2: ASTHMA_OBESITY',
  'diagnosis2: ASTHMA',
  'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS',
  'diagnosis2: OBESITY'],
 3: ['age: 56',
  'Sex: Male',
  'age: 20',
  'age: 51',
  'age: 37',
  'age: 61',
  'age: 31',
  'age: 41',
  'age: 80',
  'age: 53',
  'age: 73',
  'age: 60',
  'age: 76',
  '

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Crohn\'s Disease\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Crohn\'s Disease\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The ke

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = 1
age_row_id = 3
gender_row_id = 2

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    if value == 'primary diagnosis: HEALTHY_CONTROL':
        return '0'
    elif value == 'primary diagnosis: CROHN_DISEASE':
        return '1'
    else:
        return None



def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format



def convert_gender(value):
    # For 'gender': Binary
    if value == 'Sex: Female':
        return '1'
    elif value == 'Sex: Male':
        return '0'
    else:
        return None  # Unknown gender



In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,GSM3494893,...,GSM3495040,GSM3495041,GSM3495042,GSM3495043,GSM3495044,GSM3495045,GSM3495046,GSM3495047,GSM3495048,GSM3495049
Crohn's Disease,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Age,56,None,20,51,37,61,None,31,56,41,...,20,69,69,47,47,56,54,53,50,22
Gender,0,None,1,1,0,0,None,0,1,1,...,0,0,1,0,0,1,1,1,1,0


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,...,GSM3495040,GSM3495041,GSM3495042,GSM3495043,GSM3495044,GSM3495045,GSM3495046,GSM3495047,GSM3495048,GSM3495049
0,1,6.948572,6.783931,6.780049,7.159348,7.311038,8.522366,7.208509,7.339519,7.292977,...,6.957283,7.024298,6.953613,6.610053,6.493569,7.024804,7.469483,6.559963,6.866259,7.015812
1,2,4.267766,3.692029,3.649207,4.331090,3.903376,4.191000,3.771017,4.293601,3.963668,...,4.271395,4.137957,4.130696,3.941061,4.238553,3.597590,3.454440,3.558840,4.008234,3.686148
2,3,4.334513,3.981417,3.922257,4.549893,4.140639,4.013236,3.996610,4.933703,3.735861,...,4.303602,4.277528,4.234886,4.187910,4.143072,3.761361,3.791233,3.793128,3.344874,4.103369
3,9,7.140005,7.411580,6.722240,7.117540,6.874786,7.020385,7.031787,6.980904,7.003364,...,7.121634,6.912987,7.270386,7.561971,8.002471,7.195000,7.414168,7.845036,7.263841,7.301888
4,10,4.486670,4.615900,3.966261,4.543677,4.150289,4.216883,4.736946,4.296770,4.462828,...,4.162474,4.563109,3.977201,4.644588,4.734826,4.412295,4.422868,4.763507,5.003758,4.061337


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '9',
 '10',
 '12',
 '13',
 '14',
 '15',
 '16',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}
    


In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 489 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Crohn's Disease', the least common label is '0' with 25 occurrences. This represents 5.11% of the dataset.
The distribution of the feature 'Crohn's Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 243 occurrences. This represents 49.69% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE169568'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE169568,True,True,False,True,True,147.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [8]:
merged_data.head()

,Crohn's Disease,Age,Gender,A4GNT,AAAS,AACS,AADACL4,AAGAB,AAK1,AAMP,...,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,20,1,5.070109,5.559631,6.134333,5.031913,7.645955,8.899405,7.527557,...,6.488454,7.317771,4.979095,4.798338,5.516780,7.573893,10.875069,11.946215,9.423885,7.178631
1,0,56,1,5.425123,6.212165,6.170149,5.386187,7.792311,8.849515,8.078419,...,6.256758,7.161959,5.126632,4.644611,5.462659,7.638470,10.438330,12.496480,9.224910,6.157696
2,1,22,0,5.457190,6.376861,6.259700,4.756054,7.343049,8.507237,8.272350,...,7.066220,7.139995,4.736888,4.851393,4.883319,7.697694,10.715323,12.746669,9.165488,5.828003
3,0,32,0,5.191521,6.467095,5.896212,5.030326,7.557309,7.677507,7.833617,...,7.312751,6.971038,5.024168,4.761762,4.746781,7.239981,10.512965,12.326666,9.156230,5.973652
4,0,32,1,4.993058,6.490008,6.335348,5.092973,7.315477,7.898144,8.096650,...,6.211709,7.728567,4.706430,4.718761,5.738336,7.764994,9.825072,11.528556,9.226991,7.201866


In [9]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [10]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [11]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [12]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [13]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 43.45% ± 12.26%


In [14]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 73.10% ± 9.35%


In [15]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [16]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.1141
Found 106 genes with non-zero coefficients associated with the trait 'Crohn's Disease' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
  Variable  Coefficient  Absolute Coefficient
    FBXL17     4.280936              4.280936
     USP21     4.168490              4.168490
     WARS2     4.162085              4.162085
     PARM1    -4.070196              4.070196
     THBS1    -3.761789              3.761789
     SPDYC     3.409313              3.409313
    CCL3L3    -3.094011              3.094011
     SYTL4    -3.062172              3.062172
       HRK     3.046388              3.046388
      ADH5     3.019341              3.019341
   SULT6B1     2.832870              2.832870
    FAM72A     2.705613              2.705613
    RNF111    -2.526967              2.526967
    CLTCL1     2.523504              2.523504
    MIR877    -2.480566           

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [17]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.1141
Found 0 significant genes associated with the trait 'Crohn's Disease' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
